In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

import numpy as np


24/05/28 00:25:12 WARN Utils: Your hostname, yaozys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.2.5 instead (on interface en0)
24/05/28 00:25:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/28 00:25:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from sklearn.datasets import make_blobs

# Generate random dataset with 3 dimensions
n_samples = 100  # Number of samples
n_features = 3  # Number of dimensions
centers = 10  # Number of clusters
random_state = 42  # Random state for reproducibility

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)

In [3]:
X = X.tolist()

# 1. Construct Grid


In [4]:
# set parameters
min_pts = 5
eps = 2
dataset = X
partition_each_dim = [3 for i in range(n_features)]

# b_dataset = sc.broadcast(dataset)
# b_eps = sc.broadcast(eps)
# b_min_pts = sc.broadcast(min_pts)


In [19]:
# get the max/min bounds of each dimension
def get_minmax_by_data(dataset):
    # get the max/min bounds of each dimension
    min_max_bounds = np.concatenate(([np.min(dataset, axis=0)], [np.max(dataset, axis=0)]), axis=0).T   # (d, 2)
    return min_max_bounds

def get_minmax_by_bins(bin_ind, bin_bounds):
    min_max_bounds = []
    for i in range(len(bin_ind)):
        min_max_bounds.append((bin_bounds[i][bin_ind[i]], bin_bounds[i][bin_ind[i]+1]))
    return min_max_bounds

# get the bound of each bin
def get_bin_bounds(min_max_bounds, partition_each_dim):
    # get the bound of each bin
    # bin_bounds: list of np.array, each array contains the bounds of the bins in that dimension
    bin_bounds = []
    for i in range(len(partition_each_dim)):
        Lower_bound = min_max_bounds[i][0]
        Upper_bound = min_max_bounds[i][-1]
        dim_bins = np.linspace(Lower_bound, Upper_bound, partition_each_dim[i]+1, endpoint=True)
        bin_bounds.append(dim_bins)
    return bin_bounds

# locate the point/cell in the partitioned space, return the index of space
def find_location_id(x, bin_bounds, partition_each_dim):
    pos_list = []
    for i in range(n_features):
        # pos: the index of the bin in that dimension
        pos = np.digitize(x[i], bin_bounds[i]) - 1
        pos = min(partition_each_dim[i]-1, pos) # if the value is the max value, it should be in the last bin
        pos = max(0, pos) # if the value is the min value, it should be in the first bin
        pos_list.append(pos)
    return tuple(pos_list)


# functions for constructing grids

# calculate grid number for each dimension in one partition
def cal_grid_num(min_max_bounds, eps):
    global n_features
    grid_side_len = eps/np.sqrt(n_features)
    gridnum_each_dim = []
    for i in range(n_features):
        gridnum_each_dim.append(int((min_max_bounds[i][1] - min_max_bounds[i][0]) / grid_side_len))
    return gridnum_each_dim

# def get_grid_each_partition(pids, bin_bounds, eps):
#     grid_bin_bounds = {}
#     gridnum_each_dim = {}
#     for pid in pids:
#         gridnum_each_dim[pid] = cal_grid_num(get_minmax_by_bins(pid, bin_bounds), eps)
#         grid_bin_bounds[pid] = get_bin_bounds(get_minmax_by_bins(pid, bin_bounds), gridnum_each_dim[pid])
#     return grid_bin_bounds, gridnum_each_dim

# def find_grid(pa):
#     l = []
#     for pid, value in pa:
#         loc = find_location_id(value, grid_bin_bounds[pid], gridnum_each_dim[pid])
#         l.append((pid, loc, value))
#     return l

# def construct_grids(dataset, eps):
#     n_grid = cal_grid_num(get_minmax_by_data(dataset), eps)
#     grid_bin_bounds = get_bin_bounds(get_minmax_by_data(dataset), n_grid)



In [20]:
rdd = sc.parallelize(dataset)

# construct the grid and locate the points
min_max_bounds = get_minmax_by_data(dataset)
n_grid_each_dim = cal_grid_num(min_max_bounds, eps)
grid_bin_bounds = get_bin_bounds(min_max_bounds, n_grid_each_dim)
grid_rdd = rdd.map(lambda x: (find_location_id(x, grid_bin_bounds, n_grid_each_dim), x))
grid_rdd = grid_rdd.groupByKey().mapValues(list)
grid_rdd.cache()
grid_rdd.collect()

[((8, 14, 3),
  [[-0.290283109174704, 5.984511095595305, -6.629223876653399],
   [-0.4164968413960098, 5.902578923433743, -6.606741233991601],
   [-0.028998218638255913, 6.06053471382532, -6.69943395209346]]),
 ((8, 6, 11), [[-0.8460643599490254, -3.6616312451269524, 2.752105580753638]]),
 ((8, 11, 0),
  [[-0.025577988321223488, 2.1724577297292917, -9.201134799967729]]),
 ((15, 4, 2), [[8.545645798662383, -5.9986059024373475, -7.117236820215477]]),
 ((9, 13, 3), [[0.7538109882723547, 4.27337784989964, -6.446568843529789]]),
 ((2, 15, 10), [[-7.922925638933936, 7.652274025158388, 1.4925400310971368]]),
 ((1, 6, 6), [[-9.23526537361677, -3.970652714526209, -3.334549598894554]]),
 ((7, 17, 13), [[-1.8975213342118824, 10.045285650694272, 5.5711589553443]]),
 ((9, 11, 1),
  [[1.1842886437054831, 2.155590898117459, -8.258129626761084],
   [1.0319823733954943, 2.458661642674315, -9.091893339564193]]),
 ((5, 17, 11), [[-3.7300412730237724, 9.223149723203077, 2.6802087123483256]]),
 ((11, 2, 16

In [21]:
n_pa_each_grid = [3,3,3]
grid_min_max = np.array(([0 for _ in range(len(n_grid_each_dim))], [i-1 for i in n_grid_each_dim])).T
grid_bins = get_bin_bounds(grid_min_max, n_pa_each_grid)

grid_bins

[array([ 0.,  5., 10., 15.]),
 array([ 0.        ,  5.66666667, 11.33333333, 17.        ]),
 array([ 0.        ,  5.66666667, 11.33333333, 17.        ])]

In [51]:
from itertools import product

def find_buffer_location_id(x, bin_bounds, partition_each_dim, buffer_size):
    pos_list = []
    for i in range(n_features):
        # pos: the index of the bin in that dimension
        pos = np.digitize(x[i], bin_bounds[i]) - 1
        pos = min(partition_each_dim[i]-1, pos) # if the value is the max value, it should be in the last bin
        pos = max(0, pos) # if the value is the min value, it should be in the first bin
        
        lower_bound = bin_bounds[i][pos]
        upper_bound = bin_bounds[i][pos+1]
        pos_buffer = [pos]
        
        if x[i] - lower_bound < buffer_size:
            for j in range(1, buffer_size+1):
                pos_buffer.append(max(0, pos-j))
        if upper_bound - x[i] < buffer_size:
            for j in range(1, buffer_size+1):
                pos_buffer.append(min(partition_each_dim[i]-1, pos+j))
        pos_list.append(pos_buffer)
        
    all_combinations = list(product(*pos_list))
    return all_combinations

def add_partition_id(x, grid_bins, n_pa_each_grid, buffer_size):
    buffer_loc_id = find_buffer_location_id(x[0], grid_bins, n_pa_each_grid, buffer_size)
    buffer_loc_id = list(set(buffer_loc_id))
    return [(blid, (x[0], x[1])) for blid in buffer_loc_id]

def get_partitioned_cells(rdd, grid_bins, n_pa_each_grid, buffer_size):
    rdd1 = rdd.flatMap(lambda x: add_partition_id(x, grid_bins, n_pa_each_grid, buffer_size))
    partitioned_rdd = rdd1.groupByKey().mapValues(list)
    # partitioned_rdd = rdd1.sortByKey()
    return partitioned_rdd

buffer_size = int(np.ceil(eps/(eps/np.sqrt(n_features))))
partitioned_rdd = get_partitioned_cells(grid_rdd, grid_bins, n_pa_each_grid, buffer_size)
partitioned_rdd.glom().take(2)

# x[0] is the location id of the partitioned cell
# x[1] is the list of points in that cell, where: x[1][0] is the location id of the grid cell, x[1][1] is all the points in that grid cell


[[((1, 0, 0),
   [((9, 13, 3), [[0.7538109882723547, 4.27337784989964, -6.446568843529789]]),
    ((10, 2, 2),
     [[1.4548994656670846, -7.688120794044457, -7.381866636860483]]),
    ((10, 3, 1),
     [[1.6741623334748645, -6.787866414615768, -8.867678507876839]]),
    ((9, 12, 3),
     [[1.254433058996986, 3.7514314283377708, -6.158309451868389]]),
    ((9, 3, 3),
     [[1.1339521607180933, -7.188839567002485, -6.548846161872382]]),
    ((10, 3, 4), [[1.75349779610322, -6.522514619639524, -5.40221554853443]]),
    ((4, 6, 5),
     [[-5.444668546678066, -3.752125318334681, -3.9336470884612114]]),
    ((9, 2, 3),
     [[0.8668347099347038, -8.075833815231942, -6.067583770881749]]),
    ((9, 12, 0),
     [[0.40201615158101456, 3.1259562730292747, -9.662563134435874]]),
    ((11, 3, 3),
     [[3.3294097125115547, -6.951637312731604, -5.876576695383923]]),
    ((11, 3, 4),
     [[3.511206250406701, -6.915453230261222, -5.315465937461941]]),
    ((10, 2, 3),
     [[2.334805708988366, -7.5

In [23]:
all_inds = partitioned_rdd.keys().collect()
len(all_inds)

27

In [24]:
# transform the grid index to integer
def grid_index_mapping(n_grid_each_dim):
    grid_index = []
    for i in range(n_grid_each_dim[0]):
        for j in range(n_grid_each_dim[1]):
            for k in range(n_grid_each_dim[2]):
                grid_index.append((i, j, k))
    pos_to_gid = {}
    gid_to_pos = {}
    for n in range(len(grid_index)):
        pos_to_gid[grid_index[n]] = n
        gid_to_pos[n] = grid_index[n]
    return pos_to_gid, gid_to_pos


pos_to_gid, gid_to_pos = grid_index_mapping(n_grid_each_dim)


In [35]:
x0 = partitioned_rdd.take(1)[0][0]
x0

(1, 0, 0)

In [30]:
x1 = partitioned_rdd.take(1)[0][1]

In [32]:
x1[0][0]

(9, 13, 3)

# 2. Identify Core Point and Core cell

In [99]:
# def get_points(x):
#     grids = x[1]
#     points = []
#     for grid in grids:
#         for p in grid[1]:
#             points.append([x[0], grid[0], p])
#         # points.extend(grid[1])
#     return points

# flat_rdd = partitioned_rdd.flatMap(lambda x: get_points(x))
# # flat_rdd = partitioned_rdd.map(lambda x: get_points(x))
# flat_rdd.collect()

partitioned_rdd.collect()

[((1, 0, 0),
  [((9, 13, 3), [[0.7538109882723547, 4.27337784989964, -6.446568843529789]]),
   ((10, 2, 2),
    [[1.4548994656670846, -7.688120794044457, -7.381866636860483]]),
   ((10, 3, 1),
    [[1.6741623334748645, -6.787866414615768, -8.867678507876839]]),
   ((9, 12, 3), [[1.254433058996986, 3.7514314283377708, -6.158309451868389]]),
   ((9, 3, 3), [[1.1339521607180933, -7.188839567002485, -6.548846161872382]]),
   ((10, 3, 4), [[1.75349779610322, -6.522514619639524, -5.40221554853443]]),
   ((4, 6, 5),
    [[-5.444668546678066, -3.752125318334681, -3.9336470884612114]]),
   ((9, 2, 3), [[0.8668347099347038, -8.075833815231942, -6.067583770881749]]),
   ((9, 12, 0),
    [[0.40201615158101456, 3.1259562730292747, -9.662563134435874]]),
   ((11, 3, 3),
    [[3.3294097125115547, -6.951637312731604, -5.876576695383923]]),
   ((11, 3, 4), [[3.511206250406701, -6.915453230261222, -5.315465937461941]]),
   ((10, 2, 3),
    [[2.334805708988366, -7.5247469457563945, -6.518713987767533]]),

In [113]:
partitioned_rdd.glom().take(1)[0][1][1][0]

((1, 6, 6), [[-9.23526537361677, -3.970652714526209, -3.334549598894554]])

In [122]:
# find core points in each partition
def find_core_points(pa, eps, min_pts):
    pa = list(pa)
    for p in pa:
        pid = p[0]
        grids = p[1]
        
        points = []
        for grid in grids:
            for p in grid[1]:
                points.append([pid, grid[0], p])
                
        for pid, gid, value in points:
            count = 0
            core_flag = 0
            for _, _, value2 in points:
                if value == value2:
                    continue
                if np.linalg.norm(np.array(value) - np.array(value2)) <= eps:
                    count += 1
                    if count >= min_pts:
                        core_flag = 1
                        break
            yield ((pid, gid), value, core_flag)

core_points = partitioned_rdd.mapPartitions(lambda x: find_core_points(x, eps, min_pts))
core_points.collect()

[(((1, 0, 0), (9, 13, 3)),
  [0.7538109882723547, 4.27337784989964, -6.446568843529789],
  0),
 (((1, 0, 0), (10, 2, 2)),
  [1.4548994656670846, -7.688120794044457, -7.381866636860483],
  1),
 (((1, 0, 0), (10, 3, 1)),
  [1.6741623334748645, -6.787866414615768, -8.867678507876839],
  0),
 (((1, 0, 0), (9, 12, 3)),
  [1.254433058996986, 3.7514314283377708, -6.158309451868389],
  0),
 (((1, 0, 0), (9, 3, 3)),
  [1.1339521607180933, -7.188839567002485, -6.548846161872382],
  1),
 (((1, 0, 0), (10, 3, 4)),
  [1.75349779610322, -6.522514619639524, -5.40221554853443],
  1),
 (((1, 0, 0), (4, 6, 5)),
  [-5.444668546678066, -3.752125318334681, -3.9336470884612114],
  0),
 (((1, 0, 0), (9, 2, 3)),
  [0.8668347099347038, -8.075833815231942, -6.067583770881749],
  0),
 (((1, 0, 0), (9, 12, 0)),
  [0.40201615158101456, 3.1259562730292747, -9.662563134435874],
  0),
 (((1, 0, 0), (11, 3, 3)),
  [3.3294097125115547, -6.951637312731604, -5.876576695383923],
  0),
 (((1, 0, 0), (11, 3, 4)),
  [3.51120

In [123]:
# get the core cells

core_cells = core_points.map(lambda x: (x[0], x[2])).reduceByKey(lambda x, y: x+y).filter(lambda x: x[1] > 0)
# point_in_core_cells = core_points.join(core_cells).map(lambda x: (x[0], x[1][0]))
# point_in_core_cells.collect()
core_cells.collect()

[(((1, 0, 0), (10, 2, 3)), 1),
 (((0, 0, 1), (4, 6, 9)), 1),
 (((0, 0, 1), (2, 5, 6)), 1),
 (((1, 2, 2), (5, 17, 13)), 1),
 (((1, 2, 0), (8, 14, 3)), 3),
 (((1, 2, 0), (8, 11, 0)), 1),
 (((1, 2, 0), (9, 11, 1)), 2),
 (((1, 2, 0), (5, 16, 12)), 1),
 (((2, 1, 0), (9, 10, 0)), 2),
 (((2, 0, 0), (10, 2, 2)), 1),
 (((2, 0, 0), (14, 4, 4)), 1),
 (((0, 1, 1), (4, 6, 8)), 1),
 (((1, 2, 1), (5, 17, 12)), 1),
 (((0, 2, 1), (1, 15, 9)), 1),
 (((2, 2, 0), (9, 11, 0)), 1),
 (((1, 0, 0), (10, 2, 2)), 1),
 (((0, 0, 1), (4, 6, 8)), 1),
 (((1, 2, 2), (5, 17, 12)), 1),
 (((1, 1, 0), (9, 10, 0)), 2),
 (((1, 2, 0), (8, 13, 3)), 2),
 (((2, 1, 0), (13, 4, 3)), 1),
 (((2, 0, 0), (10, 2, 3)), 1),
 (((2, 0, 0), (13, 3, 3)), 1),
 (((2, 0, 0), (15, 4, 4)), 1),
 (((0, 1, 1), (4, 6, 9)), 1),
 (((0, 1, 1), (2, 5, 6)), 1),
 (((2, 2, 0), (9, 12, 0)), 1),
 (((0, 0, 1), (2, 6, 9)), 1),
 (((1, 2, 2), (5, 16, 12)), 1),
 (((1, 1, 0), (9, 11, 0)), 1),
 (((1, 2, 0), (8, 15, 2)), 2),
 (((0, 1, 0), (2, 5, 6)), 1),
 (((2, 1, 0

# 3. Find eps-neighbor cell pair

In [34]:
# use single linkage to find the eps-neighbor cell pair.

def find_neighbor_cells(pa, eps):
    for pid, gid, value in pa:
        for pid2, gid2, value2 in pa:
            if np.linalg.norm(np.array(value) - np.array(value2)) <= eps:
                yield ((pid, gid), (pid2, gid2))
                

# 4. Create Graph

In [35]:
# build GraphFrame for each partition
from pyspark.sql import SparkSession
from graphframes import GraphFrame


In [36]:
spark = SparkSession.builder.appName("example").getOrCreate()


In [37]:
pid = ''.join(map(str, (0,1,1)))
pid

'011'

In [38]:
def reindex(pa):
    for (pid, gid), value in pa:
        pid = ''.join(map(str, pid))
        gid = ''.join(map(str, gid))
        yield pid+gid, value
        
point_in_core_cells = point_in_core_cells.mapPartitions(reindex)
# point_in_core_cells.collect()

In [39]:
schema = "ID: STRING, features: ARRAY<FLOAT>"
v = point_in_core_cells.toDF(schema)
v.show()

+------+--------------------+
|    ID|            features|
+------+--------------------+
|011402|[-5.475511, -3.70...|
|011402|[-5.475511, -3.70...|
|021133|[-9.258973, 6.980...|
|021133|[-9.258973, 6.980...|
|021133|[-8.833215, 7.088...|
|021133|[-8.833215, 7.088...|
|122150|[-2.3378294, 8.89...|
|122150|[-2.3378294, 8.89...|
|200233|[5.7593384, -6.56...|
|200233|[5.7593384, -6.56...|
|200233|[5.411902, -7.073...|
|122040|[-3.9877195, 8.29...|
|122040|[-3.9877195, 8.29...|
|011203|[-7.4028025, -3.4...|
|011203|[-7.4028025, -3.4...|
|120322|[0.28456342, 5.71...|
|120322|[0.28456342, 5.71...|
|122140|[-2.1851137, 8.62...|
|122140|[-2.1851137, 8.62...|
|202123|[4.3889117, -8.28...|
+------+--------------------+
only showing top 20 rows



In [41]:
v.select('ID').count()

97

24/05/26 21:14:37 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 270576 ms exceeds timeout 120000 ms
24/05/26 21:14:37 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/26 21:14:42 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [40]:
# count unique id in graphframe
v.select('ID').distinct().count()

36

# 5. Compute Connect Component

# 6. Assign the border points to clusters